In [1]:
%load_ext autoreload
%autoreload 2

from preprocessing import oversample_minority_classes
import numpy as np; np.random.seed(0)
from config import OUTPUT_DIR
from util import *
from AL import AL_split, mAP_and_APs, evaluation_dict, AL_resnet16_simulation
import pickle

from config import * 
from util import *
from preprocessing import oversample_minority_classes, build_resnet16
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve
from scikeras.wrappers import KerasClassifier
from modAL import ActiveLearner
import pickle
from tensorflow import keras
import keras_cv
from keras import metrics
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score

from util import MULTICLASS_LABELS
from sklearn.metrics import average_precision_score

2024-10-03 11:30:11.647702: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 11:30:11.647750: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 11:30:11.780140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 11:30:12.039303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 11:30:13.649648: W tensorflow/comp

In [2]:
X = np.load(INTERMEDIATE / 'logmel_multiclass_noise.npy')
Y = np.load(INTERMEDIATE / 'logmel_labels_multiclass_noise.npy')

In [10]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from scipy.stats import entropy

def compute_entropy(probabilities):
    return -np.sum(probabilities * np.log(probabilities + 1e-10), axis=1)

def information_diversity(classifier, unlabeled_X, n_instances=5, alpha=1, perc=0.3):
    l = unlabeled_X.shape[0]
    
    # randomly sample a portion of X
    i = np.random.choice(range(l), size=int(perc*l))
    limited_X = unlabeled_X[i]
    
    probs = model.predict(limited_X, verbose=2)
    entropies = compute_entropy(probs)
    
    flat_X = np.array([x.flatten() for x in limited_X])
    similarities = pairwise_distances(flat_X, flat_X) # TODO differnece between unl and labeled?

    avg_similarities = np.mean(similarities, axis=1)
    
    combined_scores = entropies - alpha*avg_similarities  
    selected_indices = np.argsort(-combined_scores)[:n_instances]
    
    return limited_X[selected_indices]



In [4]:
from AL import build_resnet16

model = build_resnet16(input_shape=(40, 107, 1))

In [5]:
X.shape

(211636, 40, 107)

In [ ]:
X_c = X[:int (211636 * 0.5) ]

information_diversity(model, X_c, n_instances=5, alpha=1)

993/993 - 13s - 13ms/step
